# Stereo-seq Region Selection

## 1. Envrionment

In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import json
import gzip
import shutil
from pathlib import Path
from scipy.sparse import csr_matrix, save_npz
from scipy.io import mmwrite
from matplotlib.patches import Polygon, Rectangle
from matplotlib.transforms import Affine2D
# Use Qt5Agg for plots right now
%matplotlib qt5agg

# Find config file from the project root (assuming repo structure is as stored on GitHub)
# Change the project root path in config.yml to your local absolute location
# If you move this file, you'll need to write PROJECT_ROOT = "C:/my_path/opossum-V1-omics"
import os, sys
PROJECT_ROOT = os.path.dirname(os.getcwd())
sys.path.append(PROJECT_ROOT)
from config import root_path, dir_dict

# Set plotting parameters
sc.settings.verbosity = 3
sc.set_figure_params(dpi=100, figsize=(10, 10))
plt.rcParams['figure.figsize'] = [12, 8]

## 2. Load Data

In [2]:
# Define data path (update to your raw data directory)
data_path = dir_dict['central'] + '/samples/Stereo-seq_Mouse/'

print("Loading 10x format data...")
adata = sc.read_10x_mtx(
    data_path,
    var_names='gene_ids',  # Use gene symbols as var names
    cache=False,
    gex_only=True
)

print(f"Loaded {adata.n_obs} cells × {adata.n_vars} genes")

# Load spatial coordinates from coords.tsv.gz
print("\nLoading spatial coordinates...")
coords_file = Path(data_path) / 'coords.tsv.gz'
coords = pd.read_csv(coords_file, sep='\t')

# Match coordinates to barcodes in adata
coords = coords.set_index('barcodes')
coords = coords.reindex(adata.obs_names)

# Verify all barcodes have coordinates
if coords.isna().any().any():
    print(f"Warning: {coords.isna().sum().sum()} missing coordinate values")
    coords = coords.dropna()
    adata = adata[coords.index, :]
    print(f"Filtered to {adata.n_obs} cells with valid coordinates")

# Add coordinates to adata.obs
adata.obs['X'] = coords['X'].values * -1
adata.obs['Y'] = coords['Y'].values * -1

# Store in obsm['spatial'] for compatibility with scanpy spatial functions
adata.obsm['spatial'] = coords[['X', 'Y']].values

print(f"\nCoordinate ranges:")
print(f"  X: [{adata.obs['X'].min():.1f}, {adata.obs['X'].max():.1f}]")
print(f"  Y: [{adata.obs['Y'].min():.1f}, {adata.obs['Y'].max():.1f}]")

print("\nAnnData structure:")
print(adata)

Loading 10x format data...
--> This might be very slow. Consider passing `cache=True`, which enables much faster reading from a cache file.
Loaded 120239 cells × 61317 genes

Loading spatial coordinates...

Coordinate ranges:
  X: [-20249.0, -4064.0]
  Y: [-21728.0, -2152.0]

AnnData structure:
AnnData object with n_obs × n_vars = 120239 × 61317
    obs: 'X', 'Y'
    var: 'gene_symbols', 'feature_types'
    obsm: 'spatial'


## 3. Generate QC Metrics

In [3]:
# Calculate QC metrics if not already present
if 'n_genes_by_counts' not in adata.obs.columns:
    print("Calculating QC metrics...")
    
    # Basic QC metrics
    sc.pp.calculate_qc_metrics(adata, inplace=True)

print("\nQC Metrics Summary:")
print(f"Mean UMI per cell: {adata.obs['total_counts'].mean():.1f}")
print(f"Mean genes per cell: {adata.obs['n_genes_by_counts'].mean():.1f}")
# print(f"Mean % MT: {adata.obs['pct_counts_mt'].mean():.2f}%")

Calculating QC metrics...

QC Metrics Summary:
Mean UMI per cell: 1229.7
Mean genes per cell: 775.6


In [4]:
# Generate QC plots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Total counts distribution
axes[0, 0].hist(adata.obs['total_counts'], bins=100, color='steelblue', alpha=0.7)
axes[0, 0].set_xlabel('Total UMI counts')
axes[0, 0].set_ylabel('Number of cells')
axes[0, 0].set_title('UMI Distribution')
axes[0, 0].axvline(adata.obs['total_counts'].median(), color='red', linestyle='--', label='Median')
axes[0, 0].legend()

# Genes per spot distribution
axes[0, 1].hist(adata.obs['n_genes_by_counts'], bins=100, color='forestgreen', alpha=0.7)
axes[0, 1].set_xlabel('Genes detected')
axes[0, 1].set_ylabel('Number of cells')
axes[0, 1].set_title('Gene Detection Distribution')
axes[0, 1].axvline(adata.obs['n_genes_by_counts'].median(), color='red', linestyle='--', label='Median')
axes[0, 1].legend()

# Spatial QC plots
# Total counts spatial
scatter1 = axes[1, 0].scatter(adata.obs['Y'], adata.obs['X'], 
                              c=adata.obs['total_counts'], 
                              s=1, cmap='viridis', vmax=np.percentile(adata.obs['total_counts'], 99))
axes[1, 0].set_xlabel('X')
axes[1, 0].set_ylabel('Y')
axes[1, 0].set_aspect('equal')
plt.colorbar(scatter1, ax=axes[1, 0])

# Genes detected spatial
scatter2 = axes[1, 1].scatter(adata.obs['Y'], adata.obs['X'], 
                              c=adata.obs['n_genes_by_counts'], 
                              s=1, cmap='viridis', vmax=np.percentile(adata.obs['n_genes_by_counts'], 99))
axes[1, 1].set_xlabel('X')
axes[1, 1].set_ylabel('Y')
axes[1, 1].set_aspect('equal')
plt.colorbar(scatter2, ax=axes[1, 1])

## 4. Interactive Region Selection

This section provides tools for selecting regions and sections from the spatial data.

**Region types:**
- **Rotated Rectangle**: Define angle and bounding box coordinates or select interactively
- **Polygon**: Click to define vertices (for irregular sections)

In [4]:
class SpatialRegionSelector:
    """Interactive spatial region selection tool"""
    
    def __init__(self, adata):
        self.adata = adata
        self.selections = {'regions': [], 'sections': []}
        
    def plot_overview(self, color_by='total_counts', subsample=None):
        """Plot overview of spatial data"""
        if subsample:
            idx = np.random.choice(self.adata.n_obs, subsample, replace=False)
            x = self.adata.obs['Y'].iloc[idx]
            y = self.adata.obs['X'].iloc[idx]
            c = self.adata.obs[color_by].iloc[idx] if color_by else 'red'
        else:
            x = self.adata.obs['Y']
            y = self.adata.obs['X']
            c = self.adata.obs[color_by] if color_by else 'red'
        
        plt.figure(figsize=(12, 12))
        if color_by:
            plt.scatter(x, y, c=c, s=1, cmap='viridis', 
                       vmax=np.percentile(c, 99) if isinstance(c, pd.Series) else None)
            plt.colorbar(label=color_by)
        else:
            plt.scatter(x, y, s=1, c='red')
        
        plt.xlabel('X coordinate')
        plt.ylabel('Y coordinate')
        plt.title('Spatial Overview - Use for region selection')
        plt.axis('equal')
        plt.grid(True, alpha=0.3)
        plt.show()
        
        print(f"X range: [{x.min():.1f}, {x.max():.1f}]")
        print(f"Y range: [{y.min():.1f}, {y.max():.1f}]")
    
    def add_rotated_rectangle(self, name, angle_deg, x1, x2, y1, y2, extension_pct=0.0):
        """
        Add a rotated rectangular region
        
        Parameters:
        -----------
        name : str
            Region name (e.g., 'COL1')
        angle_deg : float
            Rotation angle in degrees
        x1, x2 : float
            X bounds in rotated space
        y1, y2 : float
            Y bounds in rotated space
        extension_pct : float
            Percentage to extend bounds (default 0.0)
        """
        # Apply extension
        if extension_pct > 0:
            y_range = y2 - y1
            extension = y_range * extension_pct
            y1 -= extension
            y2 += extension
        
        region = {
            'name': name,
            'type': 'rotated_rectangle',
            'angle_deg': angle_deg,
            'x1': x1, 'x2': x2,
            'y1': y1, 'y2': y2,
            'extension_pct': extension_pct
        }
        
        self.selections['regions'].append(region)
        print(f"Added region '{name}': angle={angle_deg}°, x=[{x1}, {x2}], y=[{y1}, {y2}]")
    
    def add_polygon_section(self, name, vertices):
        """
        Add a polygonal section
        
        Parameters:
        -----------
        name : str
            Section name (e.g., 'section1')
        vertices : list of tuples
            List of (x, y) coordinate tuples defining polygon vertices
        """
        section = {
            'name': name,
            'type': 'polygon',
            'vertices': vertices
        }
        
        self.selections['sections'].append(section)
        print(f"Added section '{name}' with {len(vertices)} vertices")
    
    def visualize_selections(self, subsample=50000, highlight_selected=False):
        """Plot all selections overlaid on spatial data"""
        # Subsample for faster plotting
        idx = np.random.choice(self.adata.n_obs, min(subsample, self.adata.n_obs), replace=False)
        x = self.adata.obs['Y'].iloc[idx]
        y = self.adata.obs['X'].iloc[idx]
        
        fig, ax = plt.subplots(figsize=(14, 14))
        
        if highlight_selected:
            # Get masks for all regions and sections
            all_masks = {}
            
            # Add region masks
            for region in self.selections['regions']:
                mask = self._get_region_mask(region)
                all_masks[region['name']] = mask
            
            # Add section masks
            for section in self.selections['sections']:
                mask = self._get_section_mask(section)
                all_masks[section['name']] = mask
            
            if all_masks:
                # Create combined mask
                combined_mask = np.zeros(self.adata.n_obs, dtype=bool)
                for mask in all_masks.values():
                    combined_mask |= mask
                
                # Plot unselected cells in gray
                unselected_idx = idx[~combined_mask[idx]]
                if len(unselected_idx) > 0:
                    ax.scatter(x.iloc[~combined_mask[idx]], y.iloc[~combined_mask[idx]], 
                            s=0.5, c='lightgray', alpha=0.3)
                
                # Plot selected cells colored by region/section
                colors = plt.cm.tab10(np.linspace(0, 1, len(all_masks)))
                for (name, mask), color in zip(all_masks.items(), colors):
                    selected_idx_mask = mask[idx]
                    if selected_idx_mask.sum() > 0:
                        ax.scatter(x.iloc[selected_idx_mask], y.iloc[selected_idx_mask], 
                                s=2, c=[color], alpha=0.8, label=f'{name}')
            else:
                ax.scatter(x, y, s=0.5, c='black', alpha=1)
        else:
            ax.scatter(x, y, s=0.5, c='black', alpha=1)
        
        # Plot regions
        for region in self.selections['regions']:
            if region['type'] == 'rotated_rectangle':
                # Define rectangle vertices in rotated space
                rect_x = [region['x1'], region['x2'], region['x2'], region['x1'], region['x1']]
                rect_y = [region['y1'], region['y1'], region['y2'], region['y2'], region['y1']]
                
                # Rotate back to original space
                theta = np.radians(-region['angle_deg'])
                cos_t, sin_t = np.cos(theta), np.sin(theta)
                rot_x = [cos_t * x - sin_t * y for x, y in zip(rect_x, rect_y)]
                rot_y = [sin_t * x + cos_t * y for x, y in zip(rect_x, rect_y)]
                
                ax.plot(rot_x, rot_y, 'r-', linewidth=2)
        
        # Plot sections
        for section in self.selections['sections']:
            if section['type'] == 'polygon':
                vertices = section['vertices']
                # Close the polygon
                x_poly = [v[0] for v in vertices] + [vertices[0][0]]
                y_poly = [v[1] for v in vertices] + [vertices[0][1]]
                ax.plot(x_poly, y_poly, 'b-', linewidth=2)
        
        ax.set_xlabel('X coordinate')
        ax.set_ylabel('Y coordinate')
        title = 'Spatial Data with Selected Regions'
        if highlight_selected:
            title += ' (Selected Cells Highlighted)'
        ax.set_title(title)
        ax.axis('equal')
        if highlight_selected:
            ax.legend()
        ax.grid(True, alpha=0.3)
        plt.show()
        
        # Print statistics if highlighting
        if highlight_selected:
            print("\nSelection Statistics:")
            for region in self.selections['regions']:
                mask = self._get_region_mask(region)
                print(f"  {region['name']}: {mask.sum():,} cells")
            for section in self.selections['sections']:
                mask = self._get_section_mask(section)
                print(f"  {section['name']}: {mask.sum():,} cells")

    def save_section_barcodes(self, output_dir=None):
        # Save selected barcodes for section(s)
        for section in self.selections['sections']:
            mask = self._get_section_mask(section)
            if mask.sum() > 0:
                adata_section = self.adata[mask].copy()
                adata_section.obs['section'] = section['name']
                
                # Save barcodes list (for compatibility with existing workflow)
                barcodes_file = output_dir / f"selected_barcodes_{section['name']}.tsv"
                pd.DataFrame(adata_section.obs_names).to_csv(barcodes_file, header=False, index=False, sep="\t")

                print(f"Saved section '{section['name']}': {mask.sum()} cells")
    
    def _get_region_mask(self, region):
        """Get boolean mask for cells in rotated rectangle region"""
        if region['type'] != 'rotated_rectangle':
            raise ValueError(f"Unknown region type: {region['type']}")
        
        # Get coordinates
        coords = np.column_stack([self.adata.obs['Y'], self.adata.obs['X']])
        
        # Apply rotation
        theta = np.radians(-region['angle_deg'])
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        rotation_matrix = np.array([[cos_t, -sin_t], [sin_t, cos_t]])
        rotated_coords = coords @ rotation_matrix.T
        
        # Check bounds
        x_rot = rotated_coords[:, 0]
        y_rot = rotated_coords[:, 1]

        mask = (
            (x_rot >= region['x1']) & (x_rot <= region['x2']) &
            (y_rot >= region['y1']) & (y_rot <= region['y2'])
        )

        return mask
    
    def _get_section_mask(self, section):
        """Get boolean mask for cells in polygon section"""
        if section['type'] != 'polygon':
            raise ValueError(f"Unknown section type: {section['type']}")
        
        from matplotlib.path import Path as MplPath
        
        # Get coordinates
        coords = np.column_stack([self.adata.obs['Y'], self.adata.obs['X']])
        
        # Create polygon path
        polygon = MplPath(section['vertices'])
        
        # Check which points are inside
        mask = polygon.contains_points(coords)
        
        return mask

# Initialize selector
selector = SpatialRegionSelector(adata)
print("Region selector initialized.")

Region selector initialized.


In [43]:
# Plot overview to have a look
selector.plot_overview(color_by='total_counts', subsample=50000)

X range: [-21728.0, -2152.0]
Y range: [-20249.0, -4064.0]


### 4a. Manual or Interactive Click-and-Drag Rectangle Selection

This section provides manual and interactive tools for selecting cortical columns:
1. Test rotation angles to align columns vertically
2. Use click-and-drag to select rectangular regions (or specify coordinates)
3. Adjust selection interactively before finalizing

In [5]:
from matplotlib.widgets import RectangleSelector

def interactive_rotated_rectangle_selector(adata, angle_deg, subsample=50000):
    """Click-and-drag rectangle selection"""
    idx = np.random.choice(adata.n_obs, min(subsample, adata.n_obs), replace=False)
    x = adata.obs['Y'].iloc[idx].values
    y = adata.obs['X'].iloc[idx].values
    
    theta = np.radians(angle_deg * -1)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    x_rot = cos_t * x - sin_t * y
    y_rot = sin_t * x + cos_t * y
    
    plt.ioff()  # Turn off interactive mode
    
    fig, ax = plt.subplots(figsize=(14, 14))
    ax.scatter(x_rot, y_rot, s=0.5, c='black', alpha=1)
    ax.set_xlabel('X (rotated)', fontsize=12)
    ax.set_ylabel('Y (rotated)', fontsize=12)
    ax.set_title(f'Rotated by {angle_deg}° - Click and drag to select region\nClose window when done',
                fontsize=14, pad=20)
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    
    bounds = {'x1': None, 'x2': None, 'y1': None, 'y2': None, 'valid': False}
    
    info_text = ax.text(0.02, 0.98, '', transform=ax.transAxes,
                       verticalalignment='top', fontsize=10,
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    def onselect(eclick, erelease):
        bounds['x1'] = min(eclick.xdata, erelease.xdata)
        bounds['x2'] = max(eclick.xdata, erelease.xdata)
        bounds['y1'] = min(eclick.ydata, erelease.ydata)
        bounds['y2'] = max(eclick.ydata, erelease.ydata)
        bounds['valid'] = True
        
        width = bounds['x2'] - bounds['x1']
        height = bounds['y2'] - bounds['y1']
        info_text.set_text(
            f"Selection:\nX: [{bounds['x1']:.1f}, {bounds['x2']:.1f}]\n"
            f"Y: [{bounds['y1']:.1f}, {bounds['y2']:.1f}]\n"
            f"Width: {width:.1f}\nHeight: {height:.1f}"
        )
        
        print(f"\nSelected: X=[{bounds['x1']:.1f}, {bounds['x2']:.1f}], "
              f"Y=[{bounds['y1']:.1f}, {bounds['y2']:.1f}]")
    
    rect_selector = RectangleSelector(
        ax, onselect,
        useblit=True,
        button=[1],
        minspanx=5, minspany=5,
        spancoords='pixels',
        interactive=True,
        props=dict(facecolor='red', edgecolor='red', alpha=0.2, fill=True)
    )
    
    print("\nClick and drag to draw rectangle. Close window when done.")
    
    plt.show(block=True)  # Block until window closed
    plt.ion()  # Turn interactive mode back on
    
    if bounds['valid']:
        return bounds['x1'], bounds['x2'], bounds['y1'], bounds['y2']
    else:
        return None

def add_rotated_rectangle_interactive(self, name, angle_deg, extension_pct=0.0):
    """Interactive click-and-drag selection"""
    print(f"\nSelecting region: {name} (angle={angle_deg}°)")
    
    result = interactive_rotated_rectangle_selector(self.adata, angle_deg)
    
    if result:
        x1, x2, y1, y2 = result
        self.add_rotated_rectangle(name, angle_deg, x1, x2, y1, y2, extension_pct)
        print(f"Region '{name}' added\n")
        return True
    else:
        print(f"Region '{name}' cancelled\n")
        return False

# Bind method to selector
selector.add_rotated_rectangle_interactive = lambda name, angle_deg, extension_pct=0.0: \
    add_rotated_rectangle_interactive(selector, name, angle_deg, extension_pct)

def preview_rotation(self, angle_deg, subsample=50000):

    """Preview data at rotation angle"""
    idx = np.random.choice(self.adata.n_obs, min(subsample, self.adata.n_obs), replace=False)
    x = adata.obs['Y'].iloc[idx].values
    y = adata.obs['X'].iloc[idx].values
    
    theta = np.radians(angle_deg * -1)
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    x_rot = cos_t * x - sin_t * y
    y_rot = sin_t * x + cos_t * y
    
    plt.figure(figsize=(10, 10))
    plt.scatter(x_rot, y_rot, s=0.5, c='black', alpha=1)
    plt.xlabel('X (rotated)')
    plt.ylabel('Y (rotated)')
    plt.title(f'Rotated by {angle_deg}°')
    plt.axis('equal')
    plt.grid(True, alpha=0.3)
    plt.show()
    
    print(f"X range: [{x_rot.min():.1f}, {x_rot.max():.1f}]")
    print(f"Y range: [{y_rot.min():.1f}, {y_rot.max():.1f}]")

selector.preview_rotation = lambda angle_deg, subsample=50000: \
    preview_rotation(selector, angle_deg, subsample)

def preview_rectangle(self, region_name=None, angle_deg=None, x1=None, x2=None, 
                     y1=None, y2=None, subsample=50000):
    """Display rectangle on original (unrotated) data"""
    
    if region_name:
        region = None
        for r in self.selections['regions']:
            if r['name'] == region_name:
                region = r
                break
        if not region:
            print(f"Region '{region_name}' not found")
            return
        angle_deg = -region['angle_deg']
        x1, x2 = region['x1'], region['x2']
        y1, y2 = region['y1'], region['y2']
    
    # Plot ORIGINAL unrotated data
    idx = np.random.choice(self.adata.n_obs, min(subsample, self.adata.n_obs), replace=False)
    x_orig = self.adata.obs['Y'].iloc[idx].values
    y_orig = self.adata.obs['X'].iloc[idx].values
    
    fig, ax = plt.subplots(figsize=(14, 14))
    ax.scatter(x_orig, y_orig, s=0.5, c='black', alpha=1)
    
    # Rectangle corners in rotated space (axis-aligned)
    rect_x_rot = [x1, x2, x2, x1, x1]
    rect_y_rot = [y1, y1, y2, y2, y1]
    
    # Rotate rectangle BACK to original space (inverse rotation)
    theta = np.radians(-angle_deg)  # Negative angle for inverse
    cos_t, sin_t = np.cos(theta), np.sin(theta)
    
    rect_x_orig = [cos_t * x - sin_t * y for x, y in zip(rect_x_rot, rect_y_rot)]
    rect_y_orig = [sin_t * x + cos_t * y for x, y in zip(rect_x_rot, rect_y_rot)]
    
    # Draw rotated rectangle on original data
    ax.plot(rect_x_orig, rect_y_orig, 'r-', linewidth=3)
    ax.fill(rect_x_orig, rect_y_orig, color='red', alpha=0.2)
    
    ax.set_xlabel('X (original)', fontsize=12)
    ax.set_ylabel('Y (original)', fontsize=12)
    title = f"Preview: {region_name}" if region_name else "Preview Rectangle"
    ax.set_title(f'{title} (rectangle rotated by {angle_deg}°)', fontsize=14, pad=20)
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    
    width = x2 - x1
    height = y2 - y1
    info_text = ax.text(0.02, 0.98, 
                       f"Selection (rotated space):\n"
                       f"X: [{x1:.1f}, {x2:.1f}]\n"
                       f"Y: [{y1:.1f}, {y2:.1f}]\n"
                       f"Width: {width:.1f}\n"
                       f"Height: {height:.1f}",
                       transform=ax.transAxes,
                       verticalalignment='top', fontsize=10,
                       bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))
    
    plt.show()
    print(f"Bounds (in rotated space): X=[{x1:.1f}, {x2:.1f}], Y=[{y1:.1f}, {y2:.1f}]")

# Update binding
selector.preview_rectangle = lambda region_name=None, angle_deg=None, x1=None, x2=None, y1=None, y2=None: \
    preview_rectangle(selector, region_name, angle_deg, x1, x2, y1, y2)

def clear_region(self, name):
    self.selections['regions'] = [r for r in self.selections['regions'] if r['name'] != name]
    print(f"Cleared region '{name}'")

selector.clear_region = lambda name: clear_region(selector, name)

print("Interactive and manual selection methods ready")

Interactive and manual selection methods ready


In [7]:
# Example: Add V1 column regions (MODIFY THESE VALUES)

# Column 1
selector.clear_region('MS_COL1')
selector.add_rotated_rectangle(
    name='MS_COL1',
    angle_deg=45,
    x1=-19450, x2=-17000,
    y1=950, y2=2500,
    extension_pct=0
)
selector.preview_rectangle(region_name='MS_COL1')

# Column 2
selector.clear_region('MS_COL2')
selector.add_rotated_rectangle(
    name='MS_COL2',
    angle_deg=10,
    x1=-17500, x2=-16300,
    y1=-3800, y2=-2400,
    extension_pct=0
)
selector.preview_rectangle(region_name='MS_COL2')

# Column 3
selector.clear_region('MS_COL3')
selector.add_rotated_rectangle(
    name='MS_COL3',
    angle_deg=-8,
    x1=-7650, x2=-6250,
    y1=-6850, y2=-5600,
    extension_pct=0
)
selector.preview_rectangle(region_name='MS_COL3')

# Column 4
selector.clear_region('MS_COL4')
selector.add_rotated_rectangle(
    name='MS_COL4',
    angle_deg=-30,
    x1=-1700, x2=-250,
    y1=-14100, y2=-12700,
    extension_pct=0
)
selector.preview_rectangle(region_name='MS_COL4')

Cleared region 'MS_COL1'
Added region 'MS_COL1': angle=45°, x=[-19450, -17000], y=[950, 2500]
Bounds (in rotated space): X=[-19450.0, -17000.0], Y=[950.0, 2500.0]
Cleared region 'MS_COL2'
Added region 'MS_COL2': angle=10°, x=[-17500, -16300], y=[-3800, -2400]
Bounds (in rotated space): X=[-17500.0, -16300.0], Y=[-3800.0, -2400.0]
Cleared region 'MS_COL3'
Added region 'MS_COL3': angle=-8°, x=[-7650, -6250], y=[-6850, -5600]
Bounds (in rotated space): X=[-7650.0, -6250.0], Y=[-6850.0, -5600.0]
Cleared region 'MS_COL4'
Added region 'MS_COL4': angle=-30°, x=[-1700, -250], y=[-14100, -12700]
Bounds (in rotated space): X=[-1700.0, -250.0], Y=[-14100.0, -12700.0]


In [8]:
# Example: Select Column 5 interactively
# Run this cell to open interactive selector
selector.clear_region('MS_COL5')
selector.add_rotated_rectangle_interactive(
    name='MS_COL5',
    angle_deg=45,
    extension_pct=0  # 10% extension on Y bounds
)
selector.preview_rectangle(region_name='MS_COL5')

Cleared region 'MS_COL5'

Selecting region: MS_COL5 (angle=45°)

Click and drag to draw rectangle. Close window when done.
Region 'MS_COL5' cancelled

Region 'MS_COL5' not found


### 4b. Define Polygon Sections (Interactive)

For whole sections, click to define vertices interactively.

In [40]:
# Interactive polygon selection
def interactive_polygon_selection(subsample=50000):
    """
    Click to define polygon vertices. Press Enter when done.
    Returns list of (x, y) tuples.
    """
    idx = np.random.choice(adata.n_obs, min(subsample, adata.n_obs), replace=False)
    x = adata.obs['Y'].iloc[idx]
    y = adata.obs['X'].iloc[idx]
    
    fig, ax = plt.subplots(figsize=(12, 12))
    ax.scatter(x, y, s=0.5, c='black', alpha=1)
    ax.set_xlabel('X coordinate')
    ax.set_ylabel('Y coordinate')
    ax.set_title('Click to define polygon vertices. Close window when done.')
    ax.axis('equal')
    ax.grid(True, alpha=0.3)
    
    vertices = []
    
    def onclick(event):
        if event.inaxes == ax:
            vertices.append((event.xdata, event.ydata))
            ax.plot(event.xdata, event.ydata, 'ro', markersize=8)
            if len(vertices) > 1:
                ax.plot([vertices[-2][0], vertices[-1][0]], 
                       [vertices[-2][1], vertices[-1][1]], 'r-', linewidth=2)
            fig.canvas.draw()
    
    cid = fig.canvas.mpl_connect('button_press_event', onclick)
    plt.show(block=True)  # Block until window closed
    
    if len(vertices) > 2:
        print(f"Selected {len(vertices)} vertices")
        return vertices
    else:
        print("Need at least 3 vertices for polygon")
        return None

In [41]:
# Section 1
vertices = interactive_polygon_selection()
if vertices:
    selector.add_polygon_section(name='MS_SEC1', vertices=vertices)

Selected 52 vertices
Added section 'MS_SEC1' with 52 vertices


## 5. Visualize and Save Selections

In [44]:
# Visualize all selections
selector.visualize_selections(subsample=50000, highlight_selected=True)


Selection Statistics:
  MS_SEC1: 31,980 cells


In [43]:
selector.save_section_barcodes(output_dir=Path(dir_dict['spatial']['seurat']['processed']))

Saved section 'MS_SEC1': 31980 cells


## 6. Generate QC Report for Selected Regions
These regions are rotated back and scaled to match sizes in the R workflow.

In [95]:
# Generate QC comparison across regions
regions = selector.selections["regions"]
n_regions = len(regions)
fig, axes = plt.subplots(2, n_regions, figsize=(5*n_regions, 10))
if n_regions == 1:
    axes = axes.reshape(-1, 1)

summary_stats = []
for idx, reg in enumerate(regions):

    name = reg['name']
    mask = selector._get_region_mask(reg)
    if mask.sum() > 0:
        adata_subset = selector.adata[mask].copy()

    # Spatial plot
    scatter = axes[0, idx].scatter(
        adata_subset.obs['X'], 
        adata_subset.obs['Y'],
        c=adata_subset.obs['total_counts'],
        s=3, cmap='viridis',
        vmax=np.percentile(adata_subset.obs['total_counts'], 99)
    )
    axes[0, idx].set_title(f'{name}\nSpatial: Total UMI')
    axes[0, idx].set_xlabel('X')
    axes[0, idx].set_ylabel('Y')
    axes[0, idx].set_aspect('equal')
    plt.colorbar(scatter, ax=axes[0, idx])
    
    # UMI distribution
    axes[1, idx].hist(adata_subset.obs['total_counts'], bins=50, 
                        color='steelblue', alpha=0.7)
    axes[1, idx].axvline(adata_subset.obs['total_counts'].median(), 
                        color='red', linestyle='--', label='Median')
    axes[1, idx].set_xlabel('Total UMI')
    axes[1, idx].set_ylabel('Number of cells')
    axes[1, idx].set_title(f'{name}\nUMI Distribution')
    axes[1, idx].legend()

    stats = {
        'Region': name,
        'N_cells': adata_subset.n_obs,
        'N_genes': adata_subset.n_vars,
        'Mean_UMI': adata_subset.obs['total_counts'].mean(),
        'Median_UMI': adata_subset.obs['total_counts'].median(),
        'Mean_genes': adata_subset.obs['n_genes_by_counts'].mean(),
        'Median_genes': adata_subset.obs['n_genes_by_counts'].median()
    }
    summary_stats.append(stats)

plt.tight_layout()
plt.show()

summary_df = pd.DataFrame(summary_stats)
print("\nRegion QC Summary:")
print(summary_df.to_string(index=False))


Region QC Summary:
 Region  N_cells  N_genes    Mean_UMI  Median_UMI  Mean_genes  Median_genes
MS_COL1     3555    61317 1380.060791      1343.0  864.671449         863.0
MS_COL2     1574    61317 1372.317627      1290.0  863.282719         836.0
MS_COL3     1909    61317 1394.485596      1285.0  882.323730         842.0
MS_COL4     2096    61317 1333.398438      1259.0  850.306775         824.0
